In [2]:
import sqlite3
import pandas as pd 
import requests

In [3]:
conn = sqlite3.connect("data/chinook.db")

***Chinook Database Skema***

![](data/chinookschema3.png)

In [4]:
# query = """
# select c.customerid, c.firstname, c.lastname,c.country, i.invoicedate ,i.total
# FROM customers as c
# LEFT JOIN invoices as i
# ON c.CustomerId = i.CustomerId
# """

In [77]:
query = """
SELECT invoices.InvoiceDate, tracks.Name as Judul
FROM invoices 
LEFT JOIN invoice_items 
ON invoices.InvoiceID = invoice_items.InvoiceID
LEFT JOIN tracks 
ON tracks.TrackId = invoice_items.TrackId 
"""

In [64]:
df = pd.read_sql_query(query,con = conn , parse_dates='InvoiceDate')
df['Bulan'] = df['InvoiceDate'].dt.month_name()
df

,InvoiceDate,Judul,Bulan
0,2009-01-01,Balls to the Wall,January
1,2009-01-01,Restless and Wild,January
2,2009-01-02,Put The Finger On You,January
3,2009-01-02,Inject The Venom,January
4,2009-01-02,Evil Walks,January
...,...,...,...
2235,2013-12-14,Looking For Love,December
2236,2013-12-14,Sweet Lady Luck,December
2237,2013-12-14,Feirinha da Pavuna/Luz do Repente/Bagaço da La...,December
2238,2013-12-14,Samba pras moças,December


In [94]:
df.isna().sum()

InvoiceDate    0
Judul          0
Bulan          0
dtype: int64

In [81]:
bulan = 'January'
condition = df['Bulan'] == bulan
a = df[condition]
a 

,InvoiceDate,Judul,Bulan
0,2009-01-01,Balls to the Wall,January
1,2009-01-01,Restless and Wild,January
2,2009-01-02,Put The Finger On You,January
3,2009-01-02,Inject The Venom,January
4,2009-01-02,Evil Walks,January
...,...,...,...
1831,2013-01-30,Pot-Pourri N.º 4,January
1832,2013-01-30,Pot-Pourri N.º 5,January
1833,2013-01-30,Por Que Será,January
1834,2013-01-30,Samba Em Prelúdio,January


In [128]:
df.melt()

,variable,value
0,InvoiceDate,2009-01-01 00:00:00
1,InvoiceDate,2009-01-01 00:00:00
2,InvoiceDate,2009-01-02 00:00:00
3,InvoiceDate,2009-01-02 00:00:00
4,InvoiceDate,2009-01-02 00:00:00
...,...,...
6715,Bulan,December
6716,Bulan,December
6717,Bulan,December
6718,Bulan,December


In [100]:
A = pd.crosstab(
    index = a['Judul'],
    columns= 'Total'
).sort_values('Total',ascending=False).head(10)
A

col_0,Total
Judul,
Powerslave,2
Dream Of Mirrors,2
Caleidoscópio,2
Iron Maiden,2
The Evil That Men Do,2
Original Of The Species,1
O Amor Nao Sabe Esperar,1
O Bicho Tá Pregando,1
O Pulso,1


KeyError: "None of ['col_0'] are in the columns"

In [86]:
A.to_json()

'{"Total":{"Powerslave":2,"Dream Of Mirrors":2,"Caleidosc\\u00f3pio":2,"Iron Maiden":2,"The Evil That Men Do":2}}'

In [5]:
# df = pd.read_sql_query(query,con = conn,parse_dates='InvoiceDate')
# df['InvoiceDOW'] = df['InvoiceDate'].dt.day_name()
# df

,CustomerId,FirstName,LastName,Country,InvoiceDate,Total,InvoiceDOW
0,1,Luís,Gonçalves,Brazil,2010-03-11,3.98,Thursday
1,1,Luís,Gonçalves,Brazil,2010-06-13,3.96,Sunday
2,1,Luís,Gonçalves,Brazil,2010-09-15,5.94,Wednesday
3,1,Luís,Gonçalves,Brazil,2011-05-06,0.99,Friday
4,1,Luís,Gonçalves,Brazil,2012-10-27,1.98,Saturday
...,...,...,...,...,...,...,...
407,59,Puja,Srivastava,India,2009-07-08,5.94,Wednesday
408,59,Puja,Srivastava,India,2010-02-26,1.99,Friday
409,59,Puja,Srivastava,India,2011-08-20,1.98,Saturday
410,59,Puja,Srivastava,India,2011-09-30,13.86,Friday


In [131]:
## gabungkan genre, tracks,album,artist,invoice items
queryy = """
SELECT artists.Name as Artist, albums.Title as albums, tracks.Name as Track, genres.Name as genre
FROM artists
LEFT JOIN albums
ON artists.ArtistId = albums.ArtistId
LEFT JOIN tracks
ON albums.AlbumId = tracks.AlbumID 
LEFT JOIN genres
ON tracks.GenreID = genres.GenreId
"""

In [132]:
df = pd.read_sql_query(queryy,conn)
df

,Artist,albums,Track,genre
0,AC/DC,For Those About To Rock We Salute You,For Those About To Rock (We Salute You),Rock
1,AC/DC,For Those About To Rock We Salute You,Put The Finger On You,Rock
2,AC/DC,For Those About To Rock We Salute You,Let's Get It Up,Rock
3,AC/DC,For Those About To Rock We Salute You,Inject The Venom,Rock
4,AC/DC,For Those About To Rock We Salute You,Snowballed,Rock
...,...,...,...,...
3569,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp","Locatelli: Concertos for Violin, Strings and C...","Concerto for Violin, Strings and Continuo in G...",Classical
3570,Emerson String Quartet,Schubert: The Late String Quartets & String Qu...,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Classical
3571,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...",Monteverdi: L'Orfeo,"L'orfeo, Act 3, Sinfonia (Orchestra)",Classical
3572,Nash Ensemble,Mozart: Chamber Music,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Classical


In [32]:
#karena kita menggunakan metode left join, maka table pertama akan dipertahankan 
#keutuhan nilainya. Sehingga ada tabel2 yang mengikuti tabel 1 akan ada kemungkinan 
#tidak memiliki nilai oleh karenanya kita akan melkukan dropna untuk mengatasi missing 
#values
d_asli = df.dropna()

In [35]:
d_asli

,Artist,albums,Track,genre
0,AC/DC,For Those About To Rock We Salute You,For Those About To Rock (We Salute You),Rock
1,AC/DC,For Those About To Rock We Salute You,Put The Finger On You,Rock
2,AC/DC,For Those About To Rock We Salute You,Let's Get It Up,Rock
3,AC/DC,For Those About To Rock We Salute You,Inject The Venom,Rock
4,AC/DC,For Those About To Rock We Salute You,Snowballed,Rock
...,...,...,...,...
3569,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp","Locatelli: Concertos for Violin, Strings and C...","Concerto for Violin, Strings and Continuo in G...",Classical
3570,Emerson String Quartet,Schubert: The Late String Quartets & String Qu...,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Classical
3571,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...",Monteverdi: L'Orfeo,"L'orfeo, Act 3, Sinfonia (Orchestra)",Classical
3572,Nash Ensemble,Mozart: Chamber Music,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Classical


In [46]:
condition = d_asli['Artist'] == 'AC/DC'
condition2 = d_asli['albums'] == 'For Those About To Rock We Salute'
d_asli[condition or condition2]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [51]:
artist = 'Nash Ensemble'
condition = d_asli['Artist'] == artist
data = d_asli[condition]
data.to_json()

'{"Artist":{"3572":"Nash Ensemble"},"albums":{"3572":"Mozart: Chamber Music"},"Track":{"3572":"Quintet for Horn, Violin, 2 Violas, and Cello in E Flat Major, K. 407\\/386c: III. Allegro"},"genre":{"3572":"Classical"}}'

end point based on dataframe above :
  -  /albumsid
  -  /artistid/album
  -  /artist/toptrack
  -  /album/track

In [52]:
import requests

In [97]:
url = "http://127.0.0.1:5000/Track/Populer/January"

In [98]:
results = requests.get(url)
data = results.json()
data = pd.DataFrame(data)
data

,Total
Caleidoscópio,2
Dream Of Mirrors,2
Iron Maiden,2
O Amor Nao Sabe Esperar,1
O Bicho Tá Pregando,1
O Pulso,1
Occupation / Precipice,1
Original Of The Species,1
Powerslave,2
The Evil That Men Do,2


In [99]:
results.json()

{'Total': {'Powerslave': 2,
  'Dream Of Mirrors': 2,
  'Caleidoscópio': 2,
  'Iron Maiden': 2,
  'The Evil That Men Do': 2,
  'Original Of The Species': 1,
  'O Amor Nao Sabe Esperar': 1,
  'O Bicho Tá Pregando': 1,
  'O Pulso': 1,
  'Occupation / Precipice': 1}}